In [1]:
import numpy as np
import pandas as pd

import scanpy
import anndata as ad

import dask
import dask.array as da

from dask_ml.decomposition import IncrementalPCA
from multiprocessing.pool import ThreadPool
from scipy.sparse import csr_matrix

import scanpy as sc

In [9]:
sc.read(f'/home/icb/jonas.flor/gastrulation_atlas/scvi/training/500k/adata_without_22.h5ad', backed='r')

AnnData object with n_obs × n_vars = 500000 × 24552 backed at '/home/icb/jonas.flor/gastrulation_atlas/scvi/training/500k/adata_without_22.h5ad'
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'batch', 'cell_cluster', 'celltype', 'cellcluster_moscot'

In [12]:
sc.read(f'/home/icb/jonas.flor/gastrulation_atlas/scvi/training/500k/10k_genes/unintegrated_adata.h5ad', backed='r')

AnnData object with n_obs × n_vars = 500000 × 24552 backed at '/home/icb/jonas.flor/gastrulation_atlas/scvi/training/500k/10k_genes/unintegrated_adata.h5ad'
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'batch', 'cell_cluster', 'celltype', 'cellcluster_moscot', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_manager_uuid', '_scvi_uuid'
    layers: 'counts'

In [2]:
dask.config.set(pool=ThreadPool(1))

data = sc.read(f'/home/icb/jonas.flor/gastrulation_atlas/scvi/training/500k/all_genes/unintegrated_adata.h5ad', backed='r').X
batch_size = 10000

def read_X(data, pos):
    return data[pos:min(pos+batch_size, data.shape[0]), :].toarray()

X = da.concatenate([da.from_delayed(dask.delayed(read_X)(data, pos), (batch_size, data.shape[1]), dtype='f4') for pos in range(0, data.shape[0], batch_size)])
X.map_blocks(csr_matrix).persist().rechunk((8192, -1))
del data

In [3]:
meta = sc.read(f'/home/icb/jonas.flor/gastrulation_atlas/scvi/training/500k/all_genes/unintegrated_adata.h5ad', backed='r')

adata = ad.AnnData(X)
del X

adata.obs = meta.obs
adata.obsm = meta.obsm
adata.layers = meta.layers
adata.uns = meta.uns
del meta

adata

AnnData object with n_obs × n_vars = 500000 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'batch', 'cell_cluster', 'celltype', 'cellcluster_moscot', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_manager_uuid', '_scvi_uuid'
    layers: 'counts'

In [4]:
dask.config.set(pool=ThreadPool(30))
n_comps = 50
pca = IncrementalPCA(n_components=n_comps, iterated_power=3)
pca = da.compute(pca.fit_transform(adata.X))

/home/icb/jonas.flor/mambaforge/envs/atlas_pca/lib/python3.10/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [5]:
adata.obsm['X_pca'] = pca[0]
del pca
adata

AnnData object with n_obs × n_vars = 500000 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'batch', 'cell_cluster', 'celltype', 'cellcluster_moscot', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_manager_uuid', '_scvi_uuid'
    obsm: 'X_pca'
    layers: 'counts'

In [6]:
sc.pp.neighbors(adata)
adata.obs.day = [float(x[1:]) for x in adata.obs.day]
adata

/home/icb/jonas.flor/mambaforge/envs/atlas_pca/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/icb/jonas.flor/mambaforge/envs/atlas_pca/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/icb/jonas.flor/mambaforge/envs/atlas_pca

AnnData object with n_obs × n_vars = 500000 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'batch', 'cell_cluster', 'celltype', 'cellcluster_moscot', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'neighbors'
    obsm: 'X_pca'
    layers: 'counts'
    obsp: 'distances', 'connectivities'